# Name Generator
This notebook shows the use of a neural network trained to generate names. Names are generated on a per-character basis based on a set of names defined in `names.txt`. The following steps are done:
- Creating a mapping for character indices
- Creating a mapping for indices to characters
- Defining x and y vectors
- Passing x and y vectors to a multi-layer perceptron model
- Generating names using the trained model

Lowercasing was not done in this notebook as the model is meant to learn the pattern of capitalizing the first letter in a name. Other forms of text cleanup, such as removing special symbols and stop words, were also not needed due to the nature of the project and the data in `names.txt`.

## Creating mappings for characters & indices

In [1]:
# Read names to use as training data
filename = "names.txt"
f = open(filename, 'r')

raw_text = f.read()

In [2]:
# Get unique characters from the training data as our vocabulary
characters = sorted(list(set(raw_text)))
print("Total Characters: {}".format(len(characters)))

Total Characters: 53


`character_indices`: Returns the index of a given character
`indices_characters`: Returns the character given an index

In [3]:
character_indices = dict((c, i) for i,c in enumerate(characters))
indices_characters = dict((i, c) for i, c in enumerate(characters))

In [4]:
character_indices

{'\n': 0,
 'A': 1,
 'B': 2,
 'C': 3,
 'D': 4,
 'E': 5,
 'F': 6,
 'G': 7,
 'H': 8,
 'I': 9,
 'J': 10,
 'K': 11,
 'L': 12,
 'M': 13,
 'N': 14,
 'O': 15,
 'P': 16,
 'Q': 17,
 'R': 18,
 'S': 19,
 'T': 20,
 'U': 21,
 'V': 22,
 'W': 23,
 'X': 24,
 'Y': 25,
 'Z': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52}

In [5]:
indices_characters

{0: '\n',
 1: 'A',
 2: 'B',
 3: 'C',
 4: 'D',
 5: 'E',
 6: 'F',
 7: 'G',
 8: 'H',
 9: 'I',
 10: 'J',
 11: 'K',
 12: 'L',
 13: 'M',
 14: 'N',
 15: 'O',
 16: 'P',
 17: 'Q',
 18: 'R',
 19: 'S',
 20: 'T',
 21: 'U',
 22: 'V',
 23: 'W',
 24: 'X',
 25: 'Y',
 26: 'Z',
 27: 'a',
 28: 'b',
 29: 'c',
 30: 'd',
 31: 'e',
 32: 'f',
 33: 'g',
 34: 'h',
 35: 'i',
 36: 'j',
 37: 'k',
 38: 'l',
 39: 'm',
 40: 'n',
 41: 'o',
 42: 'p',
 43: 'q',
 44: 'r',
 45: 's',
 46: 't',
 47: 'u',
 48: 'v',
 49: 'w',
 50: 'x',
 51: 'y',
 52: 'z'}

## Convert to a Set of Symbols of Fixed Length

`maxlen`: Dimensionality of each data point

`step`: Granularity of skips. The lower the number, the noisier. The higher the number, the more erratic.

In [6]:
maxlen = 10
step = 2

data_points = []
next_characters = []

for i in range(0, len(raw_text) - maxlen, step):
    data_points.append(raw_text[i: i+maxlen])
    next_characters.append(raw_text[i + maxlen])

In [7]:
data_points

['Michael\nCh',
 'chael\nChri',
 'ael\nChrist',
 'l\nChristop',
 'Christophe',
 'ristopher\n',
 'stopher\nJe',
 'opher\nJess',
 'her\nJessic',
 'r\nJessica\n',
 'Jessica\nMa',
 'ssica\nMatt',
 'ica\nMatthe',
 'a\nMatthew\n',
 'Matthew\nAs',
 'tthew\nAshl',
 'hew\nAshley',
 'w\nAshley\nJ',
 'Ashley\nJen',
 'hley\nJenni',
 'ey\nJennife',
 '\nJennifer\n',
 'ennifer\nJo',
 'nifer\nJosh',
 'fer\nJoshua',
 'r\nJoshua\nA',
 'Joshua\nAma',
 'shua\nAmand',
 'ua\nAmanda\n',
 '\nAmanda\nDa',
 'manda\nDani',
 'nda\nDaniel',
 'a\nDaniel\nD',
 'Daniel\nDav',
 'niel\nDavid',
 'el\nDavid\nJ',
 '\nDavid\nJam',
 'avid\nJames',
 'id\nJames\nR',
 '\nJames\nRob',
 'ames\nRober',
 'es\nRobert\n',
 '\nRobert\nJo',
 'obert\nJohn',
 'ert\nJohn\nJ',
 't\nJohn\nJos',
 'John\nJosep',
 'hn\nJoseph\n',
 '\nJoseph\nAn',
 'oseph\nAndr',
 'eph\nAndrew',
 'h\nAndrew\nR',
 'Andrew\nRya',
 'drew\nRyan\n',
 'ew\nRyan\nBr',
 '\nRyan\nBran',
 'yan\nBrando',
 'n\nBrandon\n',
 'Brandon\nJa',
 'andon\nJaso',
 'don\nJason\n',
 

In [8]:
next_characters

['r',
 's',
 'o',
 'h',
 'r',
 'J',
 's',
 'i',
 'a',
 'M',
 't',
 'h',
 'w',
 'A',
 'h',
 'e',
 '\n',
 'e',
 'n',
 'f',
 'r',
 'J',
 's',
 'u',
 '\n',
 'm',
 'n',
 'a',
 'D',
 'n',
 'e',
 '\n',
 'a',
 'i',
 '\n',
 'a',
 'e',
 '\n',
 'o',
 'e',
 't',
 'J',
 'h',
 '\n',
 'o',
 'e',
 'h',
 'A',
 'd',
 'e',
 '\n',
 'y',
 'n',
 'B',
 'a',
 'd',
 'n',
 'J',
 's',
 'n',
 'J',
 's',
 'i',
 '\n',
 'a',
 'a',
 '\n',
 'i',
 'l',
 'a',
 '\n',
 'o',
 'a',
 'h',
 'n',
 'S',
 'e',
 'h',
 'n',
 'e',
 'B',
 'i',
 'n',
 'N',
 'c',
 'l',
 '\n',
 'i',
 'h',
 'l',
 's',
 'A',
 't',
 'o',
 'y',
 'H',
 'a',
 'h',
 'r',
 'E',
 'i',
 '\n',
 'l',
 'z',
 'b',
 't',
 '\n',
 'd',
 'm',
 'M',
 'g',
 'n',
 'M',
 'l',
 's',
 'a',
 'K',
 'v',
 'n',
 'S',
 'e',
 'e',
 '\n',
 'h',
 'm',
 's',
 'T',
 'm',
 't',
 'y',
 'C',
 'r',
 's',
 'i',
 'a',
 'K',
 'l',
 '\n',
 'a',
 'h',
 'l',
 'L',
 'u',
 'a',
 'L',
 'u',
 'e',
 '\n',
 'm',
 'e',
 '\n',
 'r',
 't',
 'a',
 'y',
 'D',
 'n',
 'e',
 'l',
 '\n',
 'i',
 'h',
 'r',
 '\n

## Vectorization

In [9]:
# Convert data_points into x and characters into y
import numpy as np

x = np.zeros((len(data_points), maxlen, len(characters)), dtype=np.float64)
y = np.zeros((len(data_points), len(characters)), dtype=np.float64)

In [10]:
# Create one hot encoding of data_points and characters
# TODO: study
for i, data_point in enumerate(data_points):
    for t, character, in enumerate(data_point):
        x[i, t, character_indices[character]] = 1
    y[i, character_indices[next_characters[i]]] = 1

In [11]:
x

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [12]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# Ensure the data is on the same device to avoid errors
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
x = torch.tensor(x).float().to(device)
y = torch.tensor(y).float().to(device)

In [14]:
x.shape
# The shape of x will be: the number of data points, 
# the number of characters per data point,
# the number of characters in our vocabulary

torch.Size([66774, 10, 53])

In [15]:
# Flatten x for future operations using our model
x = torch.flatten(x, start_dim=1)
x.shape

torch.Size([66774, 530])

In [16]:
y.shape
# The shape of y will be: the number of data points,
# the number of characters in our vocabulary

torch.Size([66774, 53])

## MultiLayerPerceptron (MLP) Model

In [17]:
# Set up languge model to be used
import torch.nn as nn

class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        # Define layers and number of nodes per layer
        self.hidden = nn.Linear(input_dim, 260)
        self.output = nn.Linear(260, output_dim)
        
        # TODO: can i just merge this with the stuff below
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Define type of each layer
        x = self.relu(self.hidden(x))
        y = self.sigmoid(self.output(x))

        return y

In [18]:
# Create the model and print its architecture
model = MultiLayerPerceptron(x.shape[1], y.shape[1]).to(device)
model

MultiLayerPerceptron(
  (hidden): Linear(in_features=530, out_features=260, bias=True)
  (output): Linear(in_features=260, out_features=53, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

## Utility Function for Generating Samples

In [19]:
# Helper function to sample an index from a probability array
# TODO: study
def sample(preds, temperature=1.0):
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
# Generates sample text from a given seed, ran for every epoch of the model
# TODO: study
import random

def callback(model):
    start = 0
    stop = len(raw_text) - maxlen - 1

    print("Start: {}".format(start))
    print("Stop: {}".format(stop))

    start_index = random.randint(start, stop)

    print("Start Index: {}".format(start_index))

    sentence = raw_text[start_index: start_index + maxlen]

    print("Sentence: {}".format(sentence))
    print("Sentence Length: {}".format(len(sentence)))

    generated = ''

    for i in range(400):
        x_predictions = np.zeros((1, maxlen, len(characters)))

        for t, char in enumerate(sentence):
            x_predictions[0, t, character_indices[char]] = 1

            # print(x_predictions)
        x_predictions = torch.tensor(x_predictions).float().to(device)
        x = torch.flatten(x_predictions, start_dim=1)

        preds = model.forward(x)[0].detach().cpu().numpy()

        next_index = sample(preds)
        print("next_index: {}".format(next_index))
        next_char = indices_characters[next_index]
        print("next_char: {}".format(next_char))

        generated += next_char
        sentence = sentence[1:] + next_char

    return sentence

## Training Function

In [21]:
# Configure optimizer and loss function
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [22]:
# Function to train MLP model
import torch.nn.functional as F

def train_fn(model, optimizer, loss_fn, device):
    ave_loss = 0
    count = 0
    
    for i, data in enumerate(x):
        data = x[i]
        targets = y[i]
        
        # Forward
        predictions = model.forward(data)
        
        predictions = F.softmax(predictions, dim=-1)
        
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        count += 1
        ave_loss += loss.item()
    
    ave_loss = ave_loss / count

    return ave_loss

epochs = 10

average_losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(model, optimizer, criterion, device)
    
    average_losses.append(ave_loss)
        
    print("Ave Loss: {}".format(ave_loss))
    
    generated_sentence = callback(model)
    
    print("Generated sentence:")
    print(generated_sentence)
    print("Length: {}".format(len(generated_sentence)))

Epoch: 0
Ave Loss: 3.9601925738428316
Start: 0
Stop: 133547
Start Index: 46133
Sentence: ia
Varun
V
Sentence Length: 10
next_index: 27
next_char: a
next_index: 0
next_char: 

next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 45
next_char: s
next_index: 44
next_char: r
next_index: 38
next_char: l
next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 44
next_char: r
next_index: 31
next_char: e
next_index: 44
next_char: r
next_index: 45
next_char: s
next_index: 27
next_char: a
next_index: 34
next_char: h
next_index: 45
next_char: s
next_index: 27
next_char: a
next_index: 38
next_char: l
next_index: 41
next_char: o
next_index: 31
next_char: e
next_index: 41
next_char: o
next_index: 27
next_char: a
next_index: 38
next_char: l
next_index: 44
next_char: r
next_index: 41
next_char: o
next_index: 31
next_char: e
next_index: 46
next_char: t
next_index: 34
next_char: h
next_index: 41
next_char: o
next_index: 41

Ave Loss: 3.9566068700279162
Start: 0
Stop: 133547
Start Index: 26097
Sentence: oraine
Liz
Sentence Length: 10
next_index: 38
next_char: l
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 46
next_char: t
next_index: 27
next_char: a
next_index: 19
next_char: S
next_index: 35
next_char: i
next_index: 0
next_char: 

next_index: 27
next_char: a
next_index: 41
next_char: o
next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 34
next_char: h
next_index: 38
next_char: l
next_index: 35
next_char: i
next_index: 31
next_char: e
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 1
next_char: A
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 45
next_char: s
next_index: 0
next_char: 

next_index: 38
next_char: l
next_index: 44
next_char: r
next_index: 40
next_char: n
next_index: 0
next_char: 

next_index: 0
next_char: 

next_index: 41
next_char: o
next_index: 41
next_char: o

Ave Loss: 3.9553648066241474
Start: 0
Stop: 133547
Start Index: 33185
Sentence: estin
Jero
Sentence Length: 10
next_index: 31
next_char: e
next_index: 45
next_char: s
next_index: 35
next_char: i
next_index: 38
next_char: l
next_index: 38
next_char: l
next_index: 40
next_char: n
next_index: 45
next_char: s
next_index: 41
next_char: o
next_index: 41
next_char: o
next_index: 31
next_char: e
next_index: 35
next_char: i
next_index: 38
next_char: l
next_index: 31
next_char: e
next_index: 44
next_char: r
next_index: 0
next_char: 

next_index: 12
next_char: L
next_index: 44
next_char: r
next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 31
next_char: e
next_index: 31
next_char: e
next_index: 44
next_char: r
next_index: 0
next_char: 

next_index: 41
next_char: o
next_index: 31
next_char: e
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 10
next_char: J
next_index: 41
next_char: o
next_index: 38
next_char: l
next_index: 31
next_char: e
next_index: 40
next_char

Ave Loss: 3.9545187354641262
Start: 0
Stop: 133547
Start Index: 101309
Sentence: avon
Symeo
Sentence Length: 10
next_index: 0
next_char: 

next_index: 19
next_char: S
next_index: 31
next_char: e
next_index: 45
next_char: s
next_index: 45
next_char: s
next_index: 44
next_char: r
next_index: 40
next_char: n
next_index: 40
next_char: n
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 41
next_char: o
next_index: 45
next_char: s
next_index: 45
next_char: s
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 41
next_char: o
next_index: 35
next_char: i
next_index: 46
next_char: t
next_index: 41
next_char: o
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 12
next_char: L
next_index: 0
next_char: 

next_index: 27
next_char: a
next_index: 38
next_char: l
next_index: 31
next_char: e
next_index: 38
next_char: l
next_index: 40
next_char: n
next_index: 0
next_char: 

next_index: 4
next_char: D
next_index: 41
next_char: o
next_index: 44
next_char: 

Ave Loss: 3.953915663562731
Start: 0
Stop: 133547
Start Index: 107122
Sentence: 
Kereem
Ke
Sentence Length: 10
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 44
next_char: r
next_index: 44
next_char: r
next_index: 35
next_char: i
next_index: 45
next_char: s
next_index: 34
next_char: h
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 3
next_char: C
next_index: 40
next_char: n
next_index: 27
next_char: a
next_index: 35
next_char: i
next_index: 31
next_char: e
next_index: 35
next_char: i
next_index: 40
next_char: n
next_index: 46
next_char: t
next_index: 41
next_char: o
next_index: 35
next_char: i
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 27
next_char: a
next_index: 34
next_char: h
next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 27
next_char: a
next_index: 38
next_char: l
next_index: 40
next_char: n
next_index: 31
next_char: e
next_index: 35
next_cha

Ave Loss: 3.953515146171914
Start: 0
Stop: 133547
Start Index: 110715
Sentence: n
Deerica

Sentence Length: 10
next_index: 20
next_char: T
next_index: 44
next_char: r
next_index: 44
next_char: r
next_index: 27
next_char: a
next_index: 0
next_char: 

next_index: 10
next_char: J
next_index: 31
next_char: e
next_index: 46
next_char: t
next_index: 46
next_char: t
next_index: 41
next_char: o
next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 40
next_char: n
next_index: 44
next_char: r
next_index: 31
next_char: e
next_index: 46
next_char: t
next_index: 31
next_char: e
next_index: 44
next_char: r
next_index: 27
next_char: a
next_index: 35
next_char: i
next_index: 45
next_char: s
next_index: 27
next_char: a
next_index: 44
next_char: r
next_index: 31
next_char: e
next_index: 45
next_char: s
next_index: 35
next_char: i
next_index: 44
next_char: r
next_index: 35
next_char: i
next_index: 0
next_char: 

next_index: 18
next_char: R
next_index: 0
next_char: 

next_index: 1
next_char:

Ave Loss: 3.9532248074894065
Start: 0
Stop: 133547
Start Index: 31766
Sentence: Christon
C
Sentence Length: 10
next_index: 44
next_char: r
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 1
next_char: A
next_index: 46
next_char: t
next_index: 35
next_char: i
next_index: 38
next_char: l
next_index: 44
next_char: r
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 46
next_char: t
next_index: 0
next_char: 

next_index: 1
next_char: A
next_index: 44
next_char: r
next_index: 44
next_char: r
next_index: 38
next_char: l
next_index: 40
next_char: n
next_index: 35
next_char: i
next_index: 31
next_char: e
next_index: 38
next_char: l
next_index: 38
next_char: l
next_index: 0
next_char: 

next_index: 10
next_char: J
next_index: 0
next_char: 


Ave Loss: 3.953008045471664
Start: 0
Stop: 133547
Start Index: 56494
Sentence: ice
Jazlyn
Sentence Length: 10
next_index: 31
next_char: e
next_index: 27
next_char: a
next_index: 40
next_char: n
next_index: 0
next_char: 

next_index: 4
next_char: D
next_index: 31
next_char: e
next_index: 38
next_char: l
next_index: 0
next_char: 

next_index: 1
next_char: A
next_index: 31
next_char: e
next_index: 40
next_char: n
next_index: 46
next_char: t
next_index: 41
next_char: o
next_index: 38
next_char: l
next_index: 41
next_char: o
next_index: 27
next_char: a
next_index: 38
next_char: l
next_index: 41
next_char: o
next_index: 0
next_char: 

next_index: 19
next_char: S
next_index: 34
next_char: h
next_index: 41
next_char: o
next_index: 45
next_char: s
next_index: 34
next_char: h
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 38
next_char: l
next_index: 35
next_char: i
next_index: 27
next_char: a
next_index: 0
next_char: 

next_index: 4
next_char: D
next_index: 31
next_char: e
n

Ave Loss: 3.952842536036008
Start: 0
Stop: 133547
Start Index: 6849
Sentence: reddie
Cla
Sentence Length: 10
next_index: 40
next_char: n
next_index: 27
next_char: a
next_index: 41
next_char: o
next_index: 0
next_char: 

next_index: 1
next_char: A
next_index: 41
next_char: o
next_index: 27
next_char: a
next_index: 31
next_char: e
next_index: 35
next_char: i
next_index: 0
next_char: 

next_index: 19
next_char: S
next_index: 31
next_char: e
next_index: 0
next_char: 

next_index: 13
next_char: M
next_index: 41
next_char: o
next_index: 40
next_char: n
next_index: 0
next_char: 

next_index: 10
next_char: J
next_index: 41
next_char: o
next_index: 40
next_char: n
next_index: 35
next_char: i
next_index: 38
next_char: l
next_index: 0
next_char: 

next_index: 20
next_char: T
next_index: 35
next_char: i
next_index: 44
next_char: r
next_index: 31
next_char: e
next_index: 38
next_char: l
next_index: 0
next_char: 

next_index: 11
next_char: K
next_index: 27
next_char: a
next_index: 44
next_char: r
ne

Ave Loss: 3.952684915643834
Start: 0
Stop: 133547
Start Index: 34279
Sentence: i
Cinnamon
Sentence Length: 10
next_index: 0
next_char: 

next_index: 13
next_char: M
next_index: 41
next_char: o
next_index: 38
next_char: l
next_index: 0
next_char: 

next_index: 20
next_char: T
next_index: 31
next_char: e
next_index: 44
next_char: r
next_index: 44
next_char: r
next_index: 27
next_char: a
next_index: 0
next_char: 

next_index: 10
next_char: J
next_index: 31
next_char: e
next_index: 45
next_char: s
next_index: 41
next_char: o
next_index: 35
next_char: i
next_index: 45
next_char: s
next_index: 46
next_char: t
next_index: 0
next_char: 

next_index: 11
next_char: K
next_index: 31
next_char: e
next_index: 46
next_char: t
next_index: 46
next_char: t
next_index: 41
next_char: o
next_index: 35
next_char: i
next_index: 45
next_char: s
next_index: 0
next_char: 

next_index: 3
next_char: C
next_index: 41
next_char: o
next_index: 0
next_char: 

next_index: 10
next_char: J
next_index: 27
next_char: a
n